In [132]:
import requests
from bs4 import BeautifulSoup
import tqdm
import re
import pandas as pd

In [27]:
urls = []
for letra in tqdm.tqdm_notebook('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'):
    url = f"http://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/BuscaEmpresaListada.aspx?Letra={letra}&idioma=pt-br"
    soup = BeautifulSoup(requests.get(url).content.decode('latin1'), 'html5lib')
    urls.extend([a.attrs['href'] for a in soup.find_all('a') if "codigoCvm" in a.attrs['href']])
    

In [126]:
classificacao = {}
for url in tqdm.tqdm_notebook(urls):
    cod_cvm =  re.search("codigoCvm=(\d+)",url).expand("\\1")
    url = "http://bvmf.bmfbovespa.com.br/pt-br/mercados/acoes/empresas/ExecutaAcaoConsultaInfoEmp.asp?CodCVM=" +cod_cvm
    soup = BeautifulSoup(requests.get(url).content.decode('utf-8'), 'html5lib')
    codigos_negociacao = set((a.text for a in soup.select('a.LinkCodNeg')))
    setor = re.sub("\n|\s+",' ', soup.find('td', string='Classificação Setorial:').parent.text)
    setor = setor.replace('Classificação Setorial:','').strip()
    try:
        txt_codes = soup.select_one("#divCodigosOculto").text
        txt_codes = ' '.join(txt_codes.split())
        codigo_isin = re.search("ISIN:(.+?)\s",txt_codes).expand('\\1').strip()
    except:
        codigo_isin = None
        
    classificacao[cod_cvm] = (codigos_negociacao, setor, codigo_isin)

In [129]:
import pickle
with open('classificacao_dict.pkl','wb') as w:
    pickle.dump(classificacao,w)

In [84]:
url = "http://bvmf.bmfbovespa.com.br/pt-br/mercados/acoes/empresas/ExecutaAcaoConsultaInfoEmp.asp?CodCVM=21725"

soup = BeautifulSoup(requests.get(url).content.decode('utf-8'), 'html5lib')

In [144]:
classes_acoes = pd.Series({c:s for cs,s,i in classificacao.values() for c in cs})

In [145]:
classes_acoes = classes_acoes.str.split('/', expand=True)
classes_acoes.columns = ['Setor','Subsetor','Atividade']

In [153]:
classes_acoes.to_csv('classificacao_b3.csv')

In [77]:
url

'http://bvmf.bmfbovespa.com.br/pt-br/mercados/acoes/empresas/ExecutaAcaoConsultaInfoEmp.asp?CodCVM==21725'

In [155]:
carteiras = []
import os
import shelve
for f in set((f[:-4] for f in os.listdir() if f.startswith('resultados_'))):
    with shelve.open(f) as db:
        carteiras.append(db['carteira'])
carteiras = pd.concat(carteiras)

In [162]:
acoes = carteiras[carteiras.TP_APLIC == 'Ações']
acoes.CD_ATIVO.str.slice(0,4).dropna().unique()

array(['ABNB', 'AMBV', 'BBDC', 'BRFS', 'BTOW', 'CPLE', 'CTAX', 'GETI',
       'GGBR', 'GRND', 'HGTX', 'LAME', 'ODPV', 'PETR', 'TBLE', 'TEMP',
       'TOTS', 'VALE', 'CESP', 'GOAU', 'ITSA', 'UGPA', 'TGMA', 'PSSA',
       'CMIG', 'ELET', 'CSNA', 'CLSC', 'DASA', 'LREN', 'AMIL', 'ITUB',
       'CGAS', 'NATU', 'PNVL', 'RDCD', 'WEGE', 'ESTC', 'PRBC', 'TCSL',
       'ALPA', 'MDIA', 'ENBR', 'CSAN', 'BRTO', 'BVMF', 'ALLL', 'MULT',
       'CYRE', 'TCSA', 'RENT', 'ETER', 'ELPL', 'POMO', 'HBOR', 'RAPT',
       'UCAS', 'EQTL', 'AGRO', 'BRML', 'BRPR', 'ABEV', 'ANIM', 'ARZZ',
       'BBSE', 'CIEL', 'QUAL', 'ALSC', 'SMTO', 'BBAS', 'KROT', 'FLRY',
       'DTEX', 'CTIP', 'MILS', 'VIVT', 'LOGN', 'CPFE', 'MRFG', 'JBSS',
       'TUPY', 'FIBR', 'EVEN', 'EZTC', 'MPLU', 'TRPL', 'AMAR', 'SAPR',
       'LIGT', 'RADL', 'LINX', 'BRSR', 'MGLU', 'SEER', 'VULC', 'ENEV',
       'OMGE', 'BEEF', 'BMKS', 'CARD', 'CREM', 'HYPE', 'INPR', 'MRVE',
       'OGXP', 'RSID', 'PCAR', 'USIM', 'MRSL', 'GUAR', 'RDNI', 'LPSB',
      

In [165]:
missing = set(acoes.CD_ATIVO.str.slice(0,4).dropna().unique()) - set(classes_acoes.index.str.slice(0,4))

In [161]:
classes = pd.read_excel("classificao_acoes.xlsx", encoding='latin1')

In [172]:
missing - set(classes['CÓDIGO'])

{'ABYA',
 'AGIN',
 'ARTR',
 'BEMA',
 'BHGR',
 'CNFB',
 'ECOD',
 'INPR',
 'KSSA',
 'MARI',
 'MRSL',
 'NETC',
 'TAMM',
 'TEMP',
 'TMAR',
 'VAGR'}

In [222]:
c_final = classes_acoes[['Setor','Subsetor','Atividade']]
c_final.reset_index(inplace=True)

x = classes[['CÓDIGO','SETOR ECONÔMICO','SUBSETOR','SEGMENTO']]
x.columns = ['index','Setor','Subsetor','Atividade']

c_final = pd.concat([c_final, x])
c_final['index'] = c_final['index'].str.slice(0,4)

In [223]:
set(acoes.CD_ATIVO.str.slice(0,4).dropna().unique()) - set(c_final['index'])

{'ABYA',
 'AGIN',
 'ARTR',
 'BEMA',
 'BHGR',
 'CNFB',
 'ECOD',
 'INPR',
 'KSSA',
 'MARI',
 'MRSL',
 'NETC',
 'TAMM',
 'TEMP',
 'TMAR',
 'VAGR'}

In [227]:
c_final.drop_duplicates().set_index('index').to_csv('classes_acoes_final.csv', encoding='latin1')

In [225]:
q = classes_acoes.Subsetor.str.contains('Sid')
classes_acoes[q]

,Setor,Subsetor,Atividade
FESA4,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia
FESA3,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia
CSNA3,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia
GGBR4,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia
GGBR3,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia
MGEL4,Materiais Básicos,Siderurgia e Metalurgia,Artefatos de Ferro e Aço
MGEL3,Materiais Básicos,Siderurgia e Metalurgia,Artefatos de Ferro e Aço
GOAU4,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia
GOAU3,Materiais Básicos,Siderurgia e Metalurgia,Siderurgia
PATI4,Materiais Básicos,Siderurgia e Metalurgia,Artefatos de Ferro e Aço
